In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

## Petroleum prices

In [2]:
# read old petrol prices csv
petrol_prices = pd.read_csv('petrol_prices.csv', index_col=0, parse_dates=['period'])

# Find the starting date to get new data from the EIA API
latest_date = petrol_prices['period'].max()
next_day = latest_date + timedelta(days=1)
start_date = next_day.strftime('%Y-%m-%d')
# find the end date
today = datetime.now()
end_date = today.strftime('%Y-%m-%d')

In [3]:
# retrieve new data from the EIA API
url = f'https://api.eia.gov/v2/petroleum/pri/gnd/data/?api_key=qUwQdIbWCPzMsfeM8l9KgoATXxByk81sf3IfmugR&frequency=weekly&data[0]=value&facets[product][]=EPD2D&facets[product][]=EPMMU&facets[product][]=EPMPU&facets[product][]=EPMRU&facets[duoarea][]=NUS&facets[duoarea][]=R10&facets[duoarea][]=R1X&facets[duoarea][]=R1Y&facets[duoarea][]=R1Z&facets[duoarea][]=R20&facets[duoarea][]=R30&facets[duoarea][]=R40&facets[duoarea][]=R50&facets[duoarea][]=R5XCA&facets[duoarea][]=SCA&facets[duoarea][]=SCO&facets[duoarea][]=SFL&facets[duoarea][]=SFL&facets[duoarea][]=SMN&facets[duoarea][]=SNY&facets[duoarea][]=SOH&facets[duoarea][]=STX&facets[duoarea][]=SWA&start={start_date}&end={end_date}&sort[0][column]=period&sort[0][direction]=desc'
r = requests.get(url)
json_data1 = r.json()
temp = pd.DataFrame(json_data1['response']['data'])

In [4]:
# concatenate new data to old data
petrol_prices = pd.concat([petrol_prices, temp])
# save data

petrol_prices.to_csv('petrol_prices.csv')

## Electricity prices

In [926]:
elec_url = 'https://www.eia.gov/electricity/data/eia861m/xls/sales_revenue.xlsx'

# Get monthly residential electricity data
electricity_prices = pd.read_excel(elec_url, sheet_name='Monthly-States', usecols='A:D, H', 
                            skiprows=2, skipfooter=1, parse_dates= {"period" : ["Year","Month"]},)

us_elec = pd.read_excel(elec_url, sheet_name='US-YTD', usecols='A:C, G', 
                            skiprows=2, skipfooter=1)#, parse_dates= {"Date" : ["Year","Month"]},)


# Price in Cents/kWh
electricity_prices = electricity_prices.rename(columns={'Cents/kWh': 'price'})
us_elec = us_elec.rename(columns={'Cents/kWh': 'price'})

# Remove yearly average
us_elec = us_elec[us_elec['MONTH'] != '.']

# Change date information into Datetime
us_elec['day'] = '1'
us_elec['period'] = pd.to_datetime(us_elec[['Year', 'MONTH', 'day']])
us_elec = us_elec.drop(['Year', 'MONTH', 'day'], axis=1)

In [943]:
electricity_prices.to_csv('state_electricity.csv')
us_elec.to_csv('us_electricity.csv')

## Electricity generation CO2 emissions

#### egrid data 

data found at: https://www.epa.gov/egrid/download-data new 2021 data will be available sometime in the 1st quarter of 2023

In [385]:
# Import state annual CO2 emissions in lb/MWh
col = ['State abbreviation', 'State annual CO2 equivalent total output emission rate (lb/MWh)']
state_co2 = pd.read_excel('egrid2020_data.xlsx', sheet_name='ST20', usecols=col)
state_co2 = state_co2.drop(labels=0)
state_co2 = state_co2.rename(columns={'State abbreviation': 'state', 
                              'State annual CO2 equivalent total output emission rate (lb/MWh)': 
                                      'co2_lb/MWh'})

# Convert from lb/MWh to g/kWh
state_co2['co2_g/kWh'] = state_co2['co2_lb/MWh'] / 1000 * 453.59

In [959]:
# Import US annual CO2 emissions in lb/MWh
col = ['U.S. annual CO2 equivalent total output emission rate (lb/MWh)']
us_co2 = pd.read_excel('egrid2020_data.xlsx', sheet_name='US20', usecols=col)
us_co2 = us_co2['U.S. annual CO2 equivalent total output emission rate (lb/MWh)'].iloc[1]

# Convert from lb/MWh to g/kWh
us_co2_g_kwh = us_co2 / 1000 * 435.59

In [963]:
# Make dataframe for us data and append to state data
us_co2_all = pd.DataFrame({'state': 'US', 'co2_lb/MWh': us_co2, 'co2_g/kWh': us_co2_g_kwh, 'state_name': 'US'}, 
                         index=[53])
state_co2 = state_co2.append(us_co2_all)

# Save to csv file
state_co2.to_csv('egrid_co2_all.csv')

## Cars database

In [ ]:
# Import vehicles database
url_cars = 'https://www.fueleconomy.gov/feg/epadata/vehicles.csv'
cars = pd.read_csv(url_cars, low_memory=False)

# Save to csv file
cars.to_csv('cars_database.csv')